# Charts
_Roboquant_ notebooks come out of the box with support for several types of charts. These charts will help to better understand: 

1. what is going on during a run and evaluate the performance of the underlying strategy and policy. 
2. visualize the data feed you are using

<div class="alert alert-block alert-info">
    <b>Tip:</b> when using Jupyter Lab, roboquant will by default adapt to the configured theme. So if you select the dark theme in the settings menu of Jupyter Lab, the charts rendered by roboquant from then on will use an appropriate color scheme and everything will look great.
</div>

In [ ]:
%use roboquant(1.6.0)
Welcome()

## Price Feeds

A feed represents a stream of events you can use in your strategies. There are several type of feeds supported, like the one for Avro files used here. 

The file we are using in this notebook contains few years of S&P 500 daily prices (OHLCV). The Avro file format allows to store, access and compress data efficiently and works great for historic prices used in back-testing. 

This feed implementation also has low memory consumption since it only loads data when required. If you want to find out more about Avro files, check out their [documentation pages](https://avro.apache.org/docs/current/index.html).

In [ ]:
val feed = AvroFeed.sp500()

// We define some assets we are going to use
val apple = feed.assets.getBySymbol("AAPL")
val tesla = feed.assets.getBySymbol("TSLA")
val popularAssets = feed.assets.findBySymbols("AAPL", "GOOGL", "F", "BAC", "T", "MSFT", "CAT", "JPM", "IBM", "PFE")

// Show the first 10 assets in this feed
feed.assets.take(10).summary()

## Performance Chart
This chart enables you to see which assets got most traded and how they performed:

1. The volume decides how big the asset is plotted. The volume = (price of the asset denoted in the base currency) x (the number unit traded).
2. The return (in %) decides the color that is used. 

You can use the slider at the top to filter an a particular range of interest, for example only the big losers.

In [ ]:
val timeframe = Timeframe.fromYears(2022, 2023)
PerformanceChart(feed, timeframe = timeframe)

## Price Chart
Roboquant charts aren't designed to be a tool where you manually draw support lines on price charts to make trading decisions. 

But that doesn't mean price charts cannot come in handy. For example to validate if the data in a feed is not corrupted or has some other strange anomalies. 

There are two type of price charts:

1. PriceChart, this one support any type of pricing info

2. PriecBarChart, that only support price-bars (candlestick) data

A price chart will always plot the prices for a single asset. If we don't supply a timeframe, so all prices in the feed will be displayed. In the top right part of the plotted graph there is a toolbox that you can use to zoom into a particular area of interest.

In [ ]:
PriceChart(feed, "AAPL", timeframe = timeframe)

In [ ]:
val ind = TaLibIndicator.bbands()
val chart = PriceChart(feed, tesla, priceType = "CLOSE", indicators = arrayOf(ind))
chart.height = 900
chart

In [ ]:
val ema26 = TaLibIndicator.ema(26)
val ema12 = TaLibIndicator.ema(12)
PriceBarChart(feed, apple, indicators = arrayOf(ema12, ema26))

### Correlation Chart
This chart shows the correlation matrix of the prices of a number of assets within a feed. In this example we select some well popular stocks from different sectors and see how these are correlated.

In [ ]:
CorrelationChart(feed, popularAssets)

### Indicators
To get further insights into the price behavior of assets, you can use indicators. You can define your own, use one of the prebuilt ones like the `rsi` indicator in the example below.

In [ ]:
val rsi = TaLibIndicator.rsi(10)
val data = feed.apply(rsi, apple, tesla)
TimeSeriesChart(data)

### Strategy & Signals

Before running a back-test, you might want to validate the type of signals that your strategy is producing. You can do so, using the SignalChart. This chart plots the signals and the rating (value from -2 to 2) over time.
For this notebook we'll use the EMA crossover strategy with its default settings

In [ ]:
val strategy = EMAStrategy()
SignalChart(feed, strategy)

### Account
We run a simple strategy over the feed we just created to get some results to use for the charts. We'll also capture the AccountMetric metrics. After the run has finished, the account contains the latest state. There are several charts that provide some insights into this. 

In [ ]:
val roboquant = Roboquant(strategy, AccountMetric())
roboquant.run(feed)

## Asset Allocation Charts
Lets see what assets are in our portfolio at the end of the run. We can plot only the assets (pie chart) or plot the assets grouped per asset-class (sunburst chart). Optionally we can also say if the cash positions in the account should be included in the chart.

In [ ]:
val account = roboquant.broker.account
AllocationChart(account.positions, includeAssetClass=false)

In [ ]:
AllocationChart(account.positions, includeAssetClass=true)

In [ ]:
// You can of course also just print a summary
account.positions.summary()

## Trade Chart
Another important aspect is to see which trades has been executed as part of the run and how they performed. 

In [ ]:
val trades = account.trades.takeLast(100)
TradeChart(trades)

In [ ]:
val chart = TradeChart(trades, perAsset = true)
chart.height = 600
chart

In [ ]:
// Lets plot biggest loosing trades
val trades = account.trades.filter { it.pnl < -1000 }
TradeChart(trades, aspect = "pnl") 

In [ ]:
PriceBarChart(feed, "AAPL", account.trades)

##  Order Chart 
We can also view the orders that were created as part of the run. This time we don't plot the orders of all assets that are still in our portfolio.

In [ ]:
val assets = account.positions.assets
val orders = account.closedOrders.filter { it.asset in assets }
OrderChart(orders) 

# Metrics

Besides the charts for account related data, we can also plot the metrics that where captured during the run. Where an Account contains only the end state after the run has finished, metrics are captured during each step of the run. This allows for charts that provide more insights into different periods of the run. We used the AccountMetric metric that keeps track of how our account performs during the back-test. We use the account value (= cash + positions), for this sample

In [ ]:
val logger = roboquant.logger

// What metrics are available
logger.metricNames.summary("Metrics Names")

## Metric Chart

In [ ]:
val data = logger.getMetric("account.equity").latestRun()
TimeSeriesChart(data)

## Metric Box Chart
If you want to inspect how volatile the performance of a strategy is, the metric box chart comes in handy. It provides insights into how a metric is distributed over a period of time. If provides the following info per box:

- The minimum
- low percentile (default is 25.0)
- mid percentile (default is 50.0, aka median)
- high percentile (default is 75.0)
- the maximum 

So in this case it shows these statistics of the **daily equity returns** based on the monthly aggregate of daily values.

In [ ]:
BoxChart(data.returns())

In [ ]:
BoxChart(data.returns(), period=ChronoUnit.YEARS, lowPercentile = 10.0, highPercentile = 90.0)

## Metric Histogram Chart
If we want to see the distribution of the metric values, we can use the MetricHistogram chart. Besides the metric data that needs to be plot, you can also specify the number of bins. On the x-axis the upperbound of each bin is displayed and on the y-axis how many observations fit in that bin. 

In [ ]:
HistogramChart(data.diff(), 15)

## Calender Chart
When evaluating the performance of a strategy, it is sometimes required to find out when the strategy was performing good and bad. A calendar charts allows you to plot a metric value per day, so it is easy to find out which days to further investigate. The slider at top of the chart allows for filtering only those days that are of interest based on their value.

In the example below we plot the daily account change and loss to see when the strategy performed good (green) and when bad (red). It also demonstrates how to display metrics only for a limited timeframe.

In [ ]:
CalendarChart(data.diff())

# Multi-run tests
A good way to back-test your strategy is to run it multiple times over different time-frames. This section shows how charts that come in handy to better understand the results of those type of back-tests. 

## Walk-forward
A commonly used approach in back-testing is the so called walk-forward. The historic data is split in a number of equal periods and then the strategy is run for each period separately. This shows how the strategy would have performed during these different periods and possible different regimes.

In the following code we split the data in periods of 1 year and for each period run it. We then plot again the account value as observed during each run.

In [ ]:
roboquant.reset()

feed.timeframe.split(6.months).forEach { 
    roboquant.run(feed, it)   
}

val data = roboquant.logger.getMetric("account.equity")
TimeSeriesChart(data)

## Monte Carlo Simulation
Another great way to back-test your strategy is to run it for a fixed duration at many random starting points in time. This gives you a good insight what could be a good and bad outcome of your strategy. The following piece of code runs the strategy 25 times, each time for the duration of 6 months. Feel free to change the number of runs from 25 to a higher value like 50 or 100. It is kept small so that also people with older or less powerful computers don't have to wait too long.

In this case we plot the *account value* (= cash + positions) again and you can see there are times we do well but there are also periods when we loose money.

In [ ]:
roboquant.reset()

feed.timeframe.sample(6.months, 50).forEach {
    roboquant.run(feed, it) 
}

In [ ]:
val data = roboquant.logger.getMetric("account.equity")
TimeSeriesChart(data, useTime=false)

If you want to see the same data, but this time plotted in time, you can use the same TimeSeriesChart class and provide the *useTime=true* parameter. This shows in which periods the strategy is performing well and in which periods it is struggling. It is similar to the walk-forward approach, but provides at each moment in time multiple samples.

In [ ]:
TimeSeriesChart(data, useTime=true)

# Displaying multiple charts
By using the *chart.render()* method you can display multiple charts in the output section of a single cell. The following code selects 2 random assets from the feed and renders a candlestick chart for each one of them. It also demonstrates the excellent performance of the ECharts library, since every chart contains many data points.

In [ ]:
for (asset in popularAssets.take(2)) PriceBarChart(feed, asset).render()

Another example, lets see how the correlation between assets differs over timeframes of maximum 1 year.

In [ ]:
val assets = feed.assets.random(10)
feed.timeframe.split(1.years).forEach { 
    val chart = CorrelationChart(feed, assets, it)
    chart.title = "correlations $it"
    chart.render()
}